In [ ]:
import json
import pandas as pd

In [ ]:
with open('../../data/player_stats.json', 'r') as f:
    weekly_stats = []
    for line in f:
        weekly_stats.append(json.loads(line))
        
weekly_stats_df = pd.DataFrame(weekly_stats)

In [ ]:
weekly_stats_df['player_id'] = weekly_stats_df['url'].str.extract('([\d]+)', False)
weekly_stats_df['away_game'] = weekly_stats_df['opp'].str.contains('@')
weekly_stats_df['opp'] = weekly_stats_df['opp'].str.replace('@', '')
weekly_stats_df['date'] = pd.to_datetime(weekly_stats_df['date'] + ' 2018', format='%b %d %Y')

In [ ]:
cols_in_order = ['player_id', 'player', 'date', 'week', 'opp', 'away_game', 
                 'pass_attempts', 'pass_completions', 'pass_percent',  'pass_yards', 'pass_ya', 'pass_td', 'pass_int',
                 'rush_attempts', 'rush_yards', 'rush_avg', 'rush_td', 
                 'reception', 'rec_yards', 'rec_avg', 'rec_td', 
                 'fumb_lost', 'ko_ret_td', 'ko_ret_yards', 'punt_ret_td', 'punt_ret_yards']

if all([True if col in weekly_stats_df.columns else False for col in cols_in_order]):
    weekly_stats_df = weekly_stats_df[cols_in_order]

In [ ]:
for col in weekly_stats_df.loc['pass_attempts' : 'punt_ret_yards']:
    weekly_stats_df[col] = weekly_stats_df[col].fillna(0)

In [ ]:
weekly_stats_df.head()

In [ ]:
weekly_stats_df.to_csv('../../data/player_stats.csv', index=False)